加噪声测试

已完成：调整学习率，降低代码复杂度

In [1]:
import tqdm
from torch.nn.utils import clip_grad_norm_
from pcdet.models.detectors.pointpillar import PointPillar

import argparse
import datetime
import glob
import os
from pathlib import Path
from test import repeat_eval_ckpt, eval_single_ckpt
# from noise import add_noise_to_weights
from hardware_noise.weight_mapping_finalv import weight_mapping as add_noise_to_weights

import numba
import logging
import time

from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

import torch
import torch.distributed as dist
import torch.nn as nn
from tensorboardX import SummaryWriter

from pcdet.config import cfg, cfg_from_list, cfg_from_yaml_file, log_config_to_file
from pcdet.datasets import build_dataloader
from pcdet.models_multinomial_half import build_network, model_fn_decorator
from pcdet.utils import common_utils
from train_utils.optimization import build_optimizer, build_scheduler
# from train_utils.train_utils import train_model
# from eval_utils import eval_utils_multinomial
import numpy as np

import matplotlib.pyplot as plt
import matplotlib
from eval_utils import eval_utils_multinomial_finalv_half as eval_utils

import warnings
warnings.filterwarnings("ignore")


def load_data_to_gpu(batch_dict):
    for key, val in batch_dict.items():
        if not isinstance(val, np.ndarray):
            continue
        elif key in ['frame_id', 'metadata', 'calib']:
            continue
        elif key in ['images']:
            batch_dict[key] = image_to_tensor(val).float().cuda().contiguous()
        elif key in ['image_shape']:
            batch_dict[key] = torch.from_numpy(val).int().cuda()
        else:
            batch_dict[key] = torch.from_numpy(val).float().cuda()


def parse_config():
    parser = argparse.ArgumentParser(description='arg parser')
    parser.add_argument('--cfg_file', type=str, default='./cfgs/kitti_models/pointpillar_bayes.yaml', help='specify the config for training')
    parser.add_argument('--batch_size', type=int, default=4, required=False, help='batch size for training')
    parser.add_argument('--workers', type=int, default=32, help='number of workers for dataloader')
    parser.add_argument('--extra_tag', type=str, default='default', help='extra tag for this experiment')
    parser.add_argument('--pretrained_model', type=str, default=True, help='pretrained_model')
    parser.add_argument('--launcher', choices=['none', 'pytorch', 'slurm'], default='none')
    parser.add_argument('--tcp_port', type=int, default=18888, help='tcp port for distrbuted training')
    parser.add_argument('--sync_bn', action='store_true', default=False, help='whether to use sync bn')
    parser.add_argument('--fix_random_seed', action='store_true', default=True, help='')
    parser.add_argument('--ckpt_save_interval', type=int, default=80, help='number of training epochs')
    parser.add_argument('--local_rank', type=int, default=0, help='local rank for distributed training')
    parser.add_argument('--max_ckpt_save_num', type=int, default=81, help='max number of saved checkpoint')
    parser.add_argument('--merge_all_iters_to_one_epoch', action='store_true', default=False, help='')
    parser.add_argument('--set', dest='set_cfgs', default=None, nargs=argparse.REMAINDER, help='set extra config keys if needed')
    parser.add_argument('--max_waiting_mins', type=int, default=0, help='max waiting minutes')
    parser.add_argument('--start_epoch', type=int, default=0, help='')
    parser.add_argument('--save_to_file', action='store_true', default=False, help='')

    parser.add_argument('--epochs', type=int, default=800, required=False, help='number of epochs to train for')
    parser.add_argument('--ckpt', type=str, default='./save_path/0722/ckpt_epoch160-54.599952-61.451671.pth', help='checkpoint to start from')
    # sunqiao/OpenPCDet/tools/save_path/0722/ckpt_epoch84.pth
    args = parser.parse_known_args()[0]

    cfg_from_yaml_file(args.cfg_file, cfg)
    cfg.TAG = Path(args.cfg_file).stem
    cfg.EXP_GROUP_PATH = '/'.join(args.cfg_file.split('/')[1: -1])  # remove 'cfgs' and 'xxxx.yaml'

    if args.set_cfgs is not None:
        cfg_from_list(args.set_cfgs, cfg)

    return args, cfg




if __name__ == '__main__':
    torch.cuda.set_device(0)

    args, cfg = parse_config()
    dist_train = False
    total_gpus = 1
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.system('nvidia-smi -q -d Memory |grep -A4 GPU|grep Free >tmp')
    memory_gpu = [int(x.split()[2]) for x in open('tmp', 'r').readlines()]
    print('Using GPU:' + str(np.argmax(memory_gpu)))
    os.environ["CUDA_VISIBLE_DEVICES"] = str(np.argmax(memory_gpu))
    os.system('rm tmp')


    if args.batch_size is None:
        args.batch_size = cfg.OPTIMIZATION.BATCH_SIZE_PER_GPU
    else:
        assert args.batch_size % total_gpus == 0, 'Batch size should match the number of gpus'
        args.batch_size = args.batch_size // total_gpus

    args.epochs = cfg.OPTIMIZATION.NUM_EPOCHS if args.epochs is None else args.epochs
    
    common_utils.set_random_seed(666)
    save_path = './save_path/logger/'#/bayes/pointpillar/'+time.strftime('%m%d-%H%M',time.localtime(time.time()))+'/'

    logger = common_utils.create_logger(save_path+'log.txt', rank=cfg.LOCAL_RANK)
    
    # print("=============")
    # p1 = 0.5766
    # p2 = 0.8603
    # p3 = 0.5753
    # print(p1, p2, p3)
    # print("=============")

    # train_set, train_loader, train_sampler = build_dataloader(
    #     dataset_cfg=cfg.DATA_CONFIG,
    #     class_names=cfg.CLASS_NAMES,
    #     batch_size=args.batch_size,
    #     dist=dist_train, workers=args.workers,
    #     logger=logger,
    #     training=True,
    #     merge_all_iters_to_one_epoch=args.merge_all_iters_to_one_epoch,
    #     total_epochs=args.epochs
    # )

    test_set, test_loader, sampler = build_dataloader(
                                    dataset_cfg=cfg.DATA_CONFIG,
                                    class_names=cfg.CLASS_NAMES,
                                    batch_size=args.batch_size,
                                    dist=dist_train, workers=args.workers, logger=logger, training=False
    )

    model = PointPillar(
        model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=test_set
    )
    
    model.load_state_dict(torch.load(args.ckpt, map_location=torch.device('cuda:0')))
    # torch.load('./checkpoint_epoch_33_multinomial.pth')['model_state']
    
    model.cuda()
    

    
    # hw_data_files = sorted(os.listdir('./hardware_noise/hardware_data/'))
    file2ap_dict = {}
    N = 10
    
    f_dir = './hardware_noise/hardware_IV_and_condition_summary/hardware_data/'
    file2usability = np.load('./hardware_noise/file2usability.npy', allow_pickle=True).item()
    file2sigma = np.load('./hardware_noise/file2sigma.npy', allow_pickle=True).item()
    
    # for f_name in sorted(os.listdir(f_dir)):
    for sigma, f_name in sorted(zip(file2sigma.values(), file2sigma.keys())):
        if not f_name.endswith('.xlsx'):
            continue
        # print(f_name)
        usability = file2usability[f_name]
        # sigma = file2sigma[f_name]
        file2ap_dict[f_name] = {}
        for n in range(N):
            print('file: {}, sigma: {}, usability: {}, evaluate-{}'.format(f_name, sigma, usability, n))
            model.load_state_dict(torch.load(args.ckpt, map_location=torch.device('cuda:0')))
            add_noise_to_weights(f_dir+f_name, model, noise_sigma=sigma, device='cuda')
            
            acc1, ret_dict = eval_utils.eval_simple(
                None, None, None, f_name, usability, sigma, n, cfg, model, test_loader, logger, 
                save_path=None, dist_test=dist_train,
                save_to_file=args.save_to_file, result_dir=None
            )
            print(ret_dict)
            file2ap_dict[f_name][n] = ret_dict
                

    logger.info('**********************End evaluation**********************')    



2023-07-28 14:31:28,278   INFO  Loading KITTI dataset
2023-07-28 14:31:28,367   INFO  Total samples for KITTI dataset: 3769


Using GPU:0
file: I-V_data_1.5um_length_200nm_diameter_BT_4V_NA_third_etch_10min_Pb_ED_1h_180C_MAI_no_100nm_Ag_memory_8V_carbon_paste_no_wait_2.xlsx, sigma: 0.06283342003106338, usability: 0.25709728349414823, evaluate-0


eval: 100%|██████████| 943/943 [04:02<00:00,  3.89it/s]


{'recall/roi_0.3': 0.0, 'recall/rcnn_0.3': 0.0, 'recall/roi_0.5': 0.0, 'recall/rcnn_0.5': 0.0, 'recall/roi_0.7': 0.0, 'recall/rcnn_0.7': 0.0, 'Car_aos/easy_R40': 81.17413642832881, 'Car_aos/moderate_R40': 60.220448244064286, 'Car_aos/hard_R40': 57.27582521188113, 'Car_3d/easy_R40': 53.62619992615092, 'Car_3d/moderate_R40': 38.82763954006037, 'Car_3d/hard_R40': 36.125542680757064, 'Car_bev/easy_R40': 84.13545165784733, 'Car_bev/moderate_R40': 64.422577747004, 'Car_bev/hard_R40': 60.91943451631443, 'Car_image/easy_R40': 82.78536574940411, 'Car_image/moderate_R40': 62.83581745261938, 'Car_image/hard_R40': 60.38759723887047, 'Pedestrian_aos/easy_R40': 12.14151677393597, 'Pedestrian_aos/moderate_R40': 11.602034146692297, 'Pedestrian_aos/hard_R40': 10.538112837068384, 'Pedestrian_3d/easy_R40': 1.8626957175431165, 'Pedestrian_3d/moderate_R40': 2.0274081618662247, 'Pedestrian_3d/hard_R40': 1.8853548428729303, 'Pedestrian_bev/easy_R40': 7.5736523191511225, 'Pedestrian_bev/moderate_R40': 8.18131

eval: 100%|██████████| 943/943 [03:42<00:00,  4.24it/s]


...

{'recall/roi_0.3': 0.0, 'recall/rcnn_0.3': 0.0, 'recall/roi_0.5': 0.0, 'recall/rcnn_0.5': 0.0, 'recall/roi_0.7': 0.0, 'recall/rcnn_0.7': 0.0, 'Car_aos/easy_R40': 0.0, 'Car_aos/moderate_R40': 0.0, 'Car_aos/hard_R40': 0.0, 'Car_3d/easy_R40': 0.0, 'Car_3d/moderate_R40': 0.0, 'Car_3d/hard_R40': 0.0, 'Car_bev/easy_R40': 0.0, 'Car_bev/moderate_R40': 0.0, 'Car_bev/hard_R40': 0.0, 'Car_image/easy_R40': 0.0, 'Car_image/moderate_R40': 0.0, 'Car_image/hard_R40': 0.0, 'Pedestrian_aos/easy_R40': 0.0, 'Pedestrian_aos/moderate_R40': 0.0, 'Pedestrian_aos/hard_R40': 0.0, 'Pedestrian_3d/easy_R40': 0.0, 'Pedestrian_3d/moderate_R40': 0.0, 'Pedestrian_3d/hard_R40': 0.0, 'Pedestrian_bev/easy_R40': 0.0, 'Pedestrian_bev/moderate_R40': 0.0, 'Pedestrian_bev/hard_R40': 0.0, 'Pedestrian_image/easy_R40': 0.0, 'Pedestrian_image/moderate_R40': 0.0, 'Pedestrian_image/hard_R40': 0.0, 'Cyclist_aos/easy_R40': 0.0, 'Cyclist_aos/moderate_R40': 0.0, 'Cyclist_aos/hard_R40': 0.0, 'Cyclist_3d/easy_R40': 0.0, 'Cyclist_3d/moder

eval: 100%|██████████| 943/943 [05:58<00:00,  2.63it/s]


{'recall/roi_0.3': 0.0, 'recall/rcnn_0.3': 0.0, 'recall/roi_0.5': 0.0, 'recall/rcnn_0.5': 0.0, 'recall/roi_0.7': 0.0, 'recall/rcnn_0.7': 0.0, 'Car_aos/easy_R40': 0.0, 'Car_aos/moderate_R40': 3.2313506555585896e-05, 'Car_aos/hard_R40': 5.975217143797187e-05, 'Car_3d/easy_R40': 0.0, 'Car_3d/moderate_R40': 0.0, 'Car_3d/hard_R40': 0.0, 'Car_bev/easy_R40': 0.0, 'Car_bev/moderate_R40': 0.0, 'Car_bev/hard_R40': 0.0, 'Car_image/easy_R40': 0.0, 'Car_image/moderate_R40': 5.5020632737276485e-05, 'Car_image/hard_R40': 8.25300409349003e-05, 'Pedestrian_aos/easy_R40': 0.0, 'Pedestrian_aos/moderate_R40': 0.0, 'Pedestrian_aos/hard_R40': 0.0, 'Pedestrian_3d/easy_R40': 0.0, 'Pedestrian_3d/moderate_R40': 0.0, 'Pedestrian_3d/hard_R40': 0.0, 'Pedestrian_bev/easy_R40': 0.0, 'Pedestrian_bev/moderate_R40': 0.0, 'Pedestrian_bev/hard_R40': 0.0, 'Pedestrian_image/easy_R40': 0.0, 'Pedestrian_image/moderate_R40': 0.0, 'Pedestrian_image/hard_R40': 0.0, 'Cyclist_aos/easy_R40': 0.0, 'Cyclist_aos/moderate_R40': 0.0001

eval: 100%|█████████▉| 942/943 [04:13<00:00, 14.76it/s]2023-07-29 01:55:52,986   INFO  Average predicted number of objects(3769 samples): 49.346


ZeroDivisionError: division by zero

In [ ]:
for f_name in sorted(os.listdir('./hardware_noise/hardware_IV_and_condition_summary/hardware_data/')):
    print(f_name)